In [1]:
import sys
import os   
import traceback
import pandas as pd
import time

data_path = '../data/core_other'
data = {}
data_all = pd.DataFrame()
years = []

#final columns
columns = ['EIN', 'SUBSECCD', 'FisYr', 'NAME', 'CITY', 'STATE', 'ZIP_CODE', 'TOTAL_ASSETS', 'REVENUE', 'NET_INCOME']

data_files = os.listdir(data_path)
file_dict = dict(zip(data_files, range(len(data_files))))
print (data_files)

['core_other_1995.txt', 'core_other_1996.txt', 'core_other_1997.txt', 'core_other_1998.txt', 'core_other_1999.txt', 'core_other_2000.txt', 'core_other_2001.txt', 'core_other_2002.txt', 'core_other_2003.txt', 'core_other_2004.txt', 'core_other_2005.txt', 'core_other_2006.txt', 'core_other_2007.txt', 'core_other_2008.txt', 'core_other_2009.txt', 'core_other_2010.txt', 'core_other_2011.txt', 'core_other_2012.txt', 'core_other_2013.txt']


In [2]:
for file_name, idx in file_dict.items():
    data_file =  data_path + '/' + file_name
    
    if '~' in file_name:
        continue

    try:
        file_name_no_exnt = file_name.split('.')[0]
        year = file_name_no_exnt.split('_')[2]
        year = int(year)
    except:
        year = 'NA' 
    

    try:
        data[year] = pd.read_csv(data_file, sep=',', low_memory=False, quotechar='"')

        #convert column data into numeric type for sorting and processing
        data[year]['SUBSECCD'] = data[year]['SUBSECCD'].convert_objects(convert_numeric = True)

        #adding a common column for assets because of different columns in different year files
        if 'ASS_EOY' in data[year]:
            data[year]['TOTAL_ASSETS'] = data[year]['ASS_EOY'].convert_objects(convert_numeric = True)

        elif 'p4e_asst' in data[year]:
            data[year]['TOTAL_ASSETS'] = data[year]['p4e_asst'].convert_objects(convert_numeric = True)

        else:
            #print "TOTAL_ASSETS not found for file : " + str(file_name)
            data[year]['TOTAL_ASSETS'] = 0

        #adding a common column for revenue because of different columns in different year files
        if 'TOTREV2' in data[year]:
            data[year]['REVENUE'] = data[year]['TOTREV2'].convert_objects(convert_numeric = True)

        elif 'p1totrev' in data[year]:
            data[year]['REVENUE'] = data[year]['p1totrev'].convert_objects(convert_numeric = True)

        elif 'p1psrev' in data[year]:
            data[year]['REVENUE'] = data[year]['p1psrev'].convert_objects(convert_numeric = True)

        else:
            #print "REVENUE not found for file : " + str(file_name)
            data[year]['REVENUE'] = 0

        #adding a common column for revenue because of different columns in different year files
        if 'grossinc' in data[year]:
            data[year]['NET_INCOME'] = data[year]['grossinc'].convert_objects(convert_numeric = True)

        elif 'NETINC' in data[year]:
            data[year]['NET_INCOME'] = data[year]['NETINC'].convert_objects(convert_numeric = True)

        else:
            #print "NET_INCOME not found for file : " + str(file_name)
            data[year]['NET_INCOME'] = 0  

        #adding a common column for 5 digit zip code because of different columns in different year files for zip
        if 'ZIP5' in data[year]:
            data[year]['ZIP_CODE'] = data[year]['ZIP5'].convert_objects(convert_numeric = True)

        elif 'zip5' in data[year]:
            data[year]['ZIP_CODE'] = data[year]['zip5'].convert_objects(convert_numeric = True)

        else:
            data[year]['ZIP_CODE'] = ''

        data_all = data_all.append(data[year][columns], ignore_index=True)
        print ("loaded file : " + str(file_name))
        
    except:
        traceback.print_exc(file=sys.stdout)
        print ("error : reading files")
        exit()


loaded file : core_other_2004.txt
loaded file : core_other_2005.txt
loaded file : core_other_2006.txt
loaded file : core_other_2011.txt
loaded file : core_other_1995.txt
loaded file : core_other_2003.txt
loaded file : core_other_2012.txt
loaded file : core_other_1998.txt
loaded file : core_other_1997.txt
loaded file : core_other_1999.txt
loaded file : core_other_2000.txt
loaded file : core_other_2009.txt
loaded file : core_other_2008.txt
loaded file : core_other_2001.txt
loaded file : core_other_2013.txt
loaded file : core_other_1996.txt
loaded file : core_other_2002.txt
loaded file : core_other_2007.txt
loaded file : core_other_2010.txt
        EIN  SUBSECCD  FisYr                                          NAME  \
0  10018605         5   2004                     AMALGAMATED TRANSIT UNION   
1  10018922        19   2005                     AMERICAN LEGION AUXILIARY   
2  10018923        19   2004  AMERICAN LEGION POST 0019 THOMAS W COLE POST   
3  10018927        19   2004     AMERICAN 

In [3]:
#table 1 to get data of orgs for all years whose SUBSECCD_val = [1,12,14,15,16,50,60,80]
SUBSECCD_val = [1,12,14,15,16,50,60,80]
data_year_wise = data
data_subseccd = data_all.loc[data_all['SUBSECCD'].isin(SUBSECCD_val)]

               EIN  SUBSECCD  FisYr  \
737277   841671830         1   2007   
2367786   50620278         1   2007   
2659140   56013592         1   2009   
2659026   50204200         1   2009   
869095   470659673         1   1997   

                                                     NAME        CITY STATE  \
737277               GREATER WAYNE CNTY ECONOMIC DEV CORP         NaN    MI   
2367786           GUJARATI SOCIETY OF CENTRAL FLORIDA INC     ORLANDO    FL   
2659140            COVENTRY TEACHERS FEDERAL CREDIT UNION    COVENTRY    RI   
2659026  POSTAL GOVERNMENT EMPLOYEES FEDERAL CREDIT UNION  PROVIDENCE    RI   
869095                                  DIX FIRE DISTRICT         DIX    NE   

                ZIP  TOTAL_ASSETS  REVENUE  NET_INCOME  
737277        48226        633153   946632      569450  
2367786  32825-8408         35866    67601           3  
2659140  02816-8302       2435944    99969        8003  
2659026  02940-9606      37085275  2539987       97803  
86909

In [4]:
table_1_columns = ['SUBSECCD', 'TOTAL_ENTRIES', 'TOTAL_ASSETS', 'REVENUE', 'REVENUE_AVERAGE', 'REVENUE_MEDIAN', 'NET_INCOME', 'NET_INCOME_AVERAGE', 'NET_INCOME_MEDIAN' ]
data_table_1 = pd.DataFrame(columns=table_1_columns)
for val in SUBSECCD_val:
    data_temp = data_subseccd.loc[data_subseccd['SUBSECCD'] == val]
    total_entries = len(data_temp)
    #Total Assets
    total_asst = data_temp['TOTAL_ASSETS'].sum()
    #Revenue
    total_rev = data_temp['REVENUE'].sum()
    try:
        total_rev_mean = int(data_temp['REVENUE'].mean())
    except:
        total_rev_mean = 0
    total_rev_median = data_temp['REVENUE'].median()
    #Net Income
    total_inc = data_temp['NET_INCOME'].sum()
    try:
        total_inc_mean = int(data_temp['NET_INCOME'].mean())
    except:
        total_inc_mean = 0
    total_inc_median = data_temp['NET_INCOME'].median()        

    data_table_1.loc[SUBSECCD_val.index(val)] = [val, total_entries, total_asst, total_rev, total_rev_mean, total_rev_median, total_inc, total_inc_mean, total_inc_median]

print ("table_1")   
print (data_table_1.to_string(index=False))
data_table_1.to_csv('../results/core_other_table_1_all_years.csv', mode = 'w', index=False)    

SyntaxError: invalid syntax (<ipython-input-4-ce05cc5c9d33>, line 26)

In [22]:
#table 2 for spicific year or years
year = [2013]
data_year = pd.DataFrame()
for yr in year:
    data_year = data_year.append(data_year_wise[yr], ignore_index=True)

data_year = data_year.loc[data_year['SUBSECCD'].isin(SUBSECCD_val)]

               EIN  SUBSECCD  FisYr                           NAME  \
380840   410415960         1   1995   FIRSTEL FEDERAL CREDIT UNION   
2870447  880106922        12   1995               MT WHEELER POWER   
2870968  910145523        12   1995  BIG BEND ELECTRIC COOPERATIVE   
2870766  880331504        12   1995  SPRUCE CULINARY WATER COMPANY   
2870693  880292278        12   1995    GLENBROOK WATER COOPERATIVE   

                    CITY STATE         ZIP     ASSETS   REVENUE  
380840   BROOKLYN CENTER    MN  55430-2123  133122168  11006498  
2870447              ELY    NV  89301-1110   34155226  12659422  
2870968        RITZVILLE    WA  99169-0348   23555507  14647433  
2870766        LOGANDALE    NV  89021-0177      79759         0  
2870693        GLENBROOK    NV  89413-0447    1571276         0  


In [23]:
table_2_columns = ['SUBSECCD', 'YEAR', 'TOTAL_ENTRIES', 'TOTAL_ASSETS', 'REVENUE', 'REVENUE_AVERAGE', 'REVENUE_MEDIAN', 'NET_INCOME', 'NET_INCOME_AVERAGE', 'NET_INCOME_MEDIAN' ]
data_table_2 = pd.DataFrame(columns=table_2_columns)
for val in SUBSECCD_val:
    data_temp = data_year.loc[data_year['SUBSECCD'] == val]
    total_entries = len(data_temp)
    #Total Assets
    total_asst = data_temp['TOTAL_ASSETS'].sum()
    #Revenue
    total_rev = data_temp['REVENUE'].sum()
    try:
        total_rev_mean = int(data_temp['REVENUE'].mean())
    except:
        total_rev_mean = 0
    total_rev_median = data_temp['REVENUE'].median()
    #Net Income
    total_inc = data_temp['NET_INCOME'].sum()
    try:
        total_inc_mean = int(data_temp['NET_INCOME'].mean())
    except:
        total_inc_mean = 0
    total_inc_median = data_temp['NET_INCOME'].median()        

    data_table_2.loc[SUBSECCD_val.index(val)] = [val, year, total_entries, total_asst, total_rev, total_rev_mean, total_rev_median, total_inc, total_inc_mean, total_inc_median]

print ("table_2")   
print (data_table_2.to_string(index=False))
data_table_2.to_csv('../results/core_other_table_2_' + str(year) + '_.csv', mode = 'w', index=False)                

table_2
 SUBSECCD                                     FISYEAR  TOTAL_ENTRIES        ASSETS  ASSETS_AVERAGE  ASSETS_MEDIAN       REVENUE  REVENUE_AVERAGE  REVENUE_MEDIAN
        1  [1995, 2000, 2002, 2005, 2010, 2012, 2013]            117  1.798447e+09        15371339         165750  1.201346e+08          1026791         90609.0
       12  [1995, 2000, 2002, 2005, 2010, 2012, 2013]          26471  6.614977e+11        24989525        1320249  2.877077e+11         10868789        286117.0
       14  [1995, 2000, 2002, 2005, 2010, 2012, 2013]          14051  2.374833e+12       169015195       17511855  1.330120e+11          9467046        976967.5
       15  [1995, 2000, 2002, 2005, 2010, 2012, 2013]           4933  2.034651e+10         4124570         651653  3.484695e+09           706404        168203.0
       16  [1995, 2000, 2002, 2005, 2010, 2012, 2013]             89  3.035230e+09        34103711       10905086  1.852457e+08          2081412       1468180.0
       50  [1995, 2000, 20

In [24]:
#table 3 for specific state or states
state = ['CA']
data_state = data_all.loc[data_all['STATE'].isin(state)]
data_state = data_state.loc[data_state['SUBSECCD'].isin(SUBSECCD_val)]

               EIN  SUBSECCD  FisYr  \
1979494  237443056         1   2007   
141477   946102010        12   2005   
141465   946101671        12   2005   
141459   946101177        12   2006   
141455   946100221        12   2005   

                                                NAME        CITY STATE  \
1979494  CITY OF MANTECA RECREATIONAL FACILITIES INC         NaN    CA   
141477                  BRANGER MUTUAL WATER COMPANY  SANTA ROSA    CA   
141465                    DELTA MUTUAL WATER COMPANY      OAKLEY    CA   
141459            PARADISE LAKE MUTUAL WATER COMPANY   PRUNEDALE    CA   
141455             KINGS MOUNTAIN PARK WATER COMPANY    WOODSIDE    CA   

                ZIP  ASSETS  REVENUE  
1979494       95337   28326    67645  
141477   95405-0354  401815    54744  
141465   94561-5027  126671    37451  
141459   93907-8719       0    52611  
141455   94062-4586   14473    25792  


In [25]:
table_3_columns = ['SUBSECCD', 'STATE', 'TOTAL_ENTRIES', 'ASSETS', 'REVENUE', 'REVENUE_AVERAGE', 'REVENUE_MEDIAN', 'NET_INCOME', 'NET_INCOME_AVERAGE', 'NET_INCOME_MEDIAN' ]
data_table_3 = pd.DataFrame(columns=table_3_columns)
for val in SUBSECCD_val:
    data_temp = data_state.loc[data_state['SUBSECCD'] == val]
    total_entries = len(data_temp)
    #Total Assets
    total_asst = data_temp['TOTAL_ASSETS'].sum()
    #Revenue
    total_rev = data_temp['REVENUE'].sum()
    try:
        total_rev_mean = int(data_temp['REVENUE'].mean())
    except:
        total_rev_mean = 0
    total_rev_median = data_temp['REVENUE'].median()
    #Net Income
    total_inc = data_temp['NET_INCOME'].sum()
    try:
        total_inc_mean = int(data_temp['NET_INCOME'].mean())
    except:
        total_inc_mean = 0
    total_inc_median = data_temp['NET_INCOME'].median()        

    data_table_3.loc[SUBSECCD_val.index(val)] = [val, state, total_entries, total_asst, total_rev, total_rev_mean, total_rev_median, total_inc, total_inc_mean, total_inc_median]

print ("table_3")      
print (data_table_3.to_string(index=False))
data_table_3.to_csv('../results/core_other_table_3_' + str(state) + '_.csv', mode = 'w', index=False)        

table_3
 SUBSECCD             STATE  TOTAL_ENTRIES        ASSETS  ASSETS_AVERAGE  ASSETS_MEDIAN      REVENUE  REVENUE_AVERAGE  REVENUE_MEDIAN
        1  [MA, NY, CA, OR]             23  9.398873e+06          408646        28326.0      5408396           235147           57824
       12  [MA, NY, CA, OR]           9510  2.898248e+10         3047579       262535.5  10496984668          1103783           84558
       14  [MA, NY, CA, OR]           3636  1.531210e+12       421124729     46892662.0  92120591555         25335696         2707902
       15  [MA, NY, CA, OR]           1505  1.575060e+10        10465511       414621.0   2790745912          1854316          157353
       16  [MA, NY, CA, OR]             27  6.480350e+08        24001296     31114667.0     31390999          1162629         1479099
       50  [MA, NY, CA, OR]             36  5.204746e+07         1445762       447654.0     34805706           966825           74080
       60  [MA, NY, CA, OR]              0  0.000000e+

In [ ]:
#map plotting
SUBSECCD_val = [1,12,14,15,16,50,60,80]

subseccd = {}
for val in SUBSECCD_val:
    subseccd[val] = data_all.loc[data_all['SUBSECCD'].isin([val])]


for val in SUBSECCD_val:
    print list(subseccd[val]['EIN'].values)
    print list(subseccd[val]['ZIP_CODE'].values)